<a href="https://colab.research.google.com/github/wbendinelli/SnapScrub/blob/main/beta_photograph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Selection and Evaluation Pipeline

## Objective:
----------
This notebook implements a comprehensive pipeline for automated image processing, aiming to:
1. Organize and preprocess image files systematically.
2. Rename and resize images for consistent analysis.
3. Detect and remove duplicate images using Structural Similarity Index (SSIM).
4. Predict image popularity using pre-trained Convolutional Neural Networks (CNNs).
5. Generate a final report with computed scores for each processed image.


## Techniques and Methods Used:
----------------------------
1. **Initial Setup**:
   - **Google Drive Integration**: If running in Google Colab, the notebook automatically mounts Google Drive to access and store files.
   - **Directory Management**: Dynamically creates necessary directories for raw images, processed images, and results.
2. **Preprocessing**:
   - **Renaming**: Sequentially renames images to ensure consistency and traceability.
   - **Resizing**: Uses the Python Imaging Library (Pillow) to resize all images to a standard size of 256x256 pixels, optimizing them for further processing.
3. **Duplicate Removal**:
   - **SSIM**: Utilizes the Structural Similarity Index (from `skimage`) to compare grayscale versions of images. A threshold (default: 0.9) is used to identify and remove duplicates.
4. **Popularity Prediction**:
   - **Pre-trained CNNs**: Leverages models such as MobileNetV2, InceptionV3, ResNet50, and EfficientNetB0, available from TensorFlow's Keras Applications.
   - **Feature Extraction**: Extracts high-level image features using global average pooling layers of the CNNs.
   - **Cosine Similarity**: Calculates similarity scores between extracted features and a predefined vector, serving as a proxy for social media popularity.
5. **Results**:
   - **Data Aggregation**: Stores predictions in a structured format (CSV) for analysis and visualization.
   - **Summary Statistics**: Computes descriptive statistics (mean, max, min) of the scores for insights.

# Library imports

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from google.colab import drive
from PIL import Image
from tensorflow.keras.applications import MobileNetV2, InceptionV3, ResNet50, EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from sklearn.metrics.pairwise import cosine_similarity
import logging
from tqdm import tqdm_notebook as tqdm

In [4]:
import logging

# Configure logging explicitly for Colab
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Clear existing handlers
if logger.hasHandlers():
    logger.handlers.clear()

# Add a stream handler to print logs to the console
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [5]:
def mount_google_drive():
    """
    Mount Google Drive in the Colab environment.

    This function mounts Google Drive at the default location (`/content/drive`),
    allowing access to files stored in the user's Drive. If the Drive is already
    mounted, it will not remount unless explicitly forced using `force_remount=True`.

    Logs a success message if the Drive is mounted successfully, or an error message
    if the operation fails.

    Raises:
        Exception: If the Drive cannot be mounted due to an error.
    """
    try:
        drive.mount('/content/drive', force_remount=False)
        logging.info("Google Drive is mounted and ready to use.")
    except Exception as e:
        logging.error(f"Failed to mount Google Drive: {e}")
        raise

In [6]:
mount_google_drive()

2025-01-19 16:26:00,742 - INFO - Google Drive is mounted and ready to use.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
def create_folders(root_path):
    """
    Create the necessary directory structure for the image classification pipeline.

    This function ensures the following subdirectories are created under the specified root path:
    - `original`: Contains the original images before any processing (untouched).
    - `resized`: Contains resized images for analysis.
    - `cleaned`: Contains cleaned images (e.g., without duplicates or corrupt files).
    - `results`: Stores intermediate and final results, such as evaluation tables or reports.

    Deletes all existing files and directories in the root path to ensure a clean slate, including in the 'original' folder.

    Parameters:
        root_path (str): The root directory where the subdirectories will be created.

    Returns:
        dict: A dictionary containing the names of the subdirectories as keys and their full paths as values.
    """
    # Delete all existing files and directories in the root path
    if os.path.exists(root_path):
        for item in os.listdir(root_path):
            item_path = os.path.join(root_path, item)
            try:
                if os.path.isfile(item_path) or os.path.islink(item_path):
                    os.remove(item_path)
                elif os.path.isdir(item_path):
                    shutil.rmtree(item_path)
            except Exception as e:
                logging.error(f"Error deleting {item_path}: {e}")

    # Create necessary subdirectories
    folders = {
        "original": os.path.join(root_path, "original"),
        "resized": os.path.join(root_path, "resized"),
        "converted": os.path.join(root_path, "converted"),
        "cleaned": os.path.join(root_path, "cleaned"),
        "results": os.path.join(root_path, "results"),
    }

    for path in folders.values():
        os.makedirs(path, exist_ok=True)

    logging.info(f"Project initialized with folders: {list(folders.values())}")
    return folders

def copy_to_original(source_path, target_path):
    """
    Copy files from a source directory to the 'original' folder.

    This function copies all files from the source directory to the 'original' folder
    created by the `create_folders` function. Existing files in the target folder will
    be replaced.

    Parameters:
        source_path (str): The directory containing files to be copied.
        target_path (str): The destination 'original' folder.

    Returns:
        list: A list of file names successfully copied.
    """
    if not os.path.exists(source_path):
        logging.error(f"Source path does not exist: {source_path}")
        return []

    if not os.path.exists(target_path):
        logging.error(f"Target path does not exist: {target_path}")
        return []

    copied_files = []
    errors = []

    for file_name in os.listdir(source_path):
        source_file = os.path.join(source_path, file_name)
        target_file = os.path.join(target_path, file_name)

        # Only copy files, skip directories
        if os.path.isfile(source_file):
            try:
                shutil.copy2(source_file, target_file)
                copied_files.append(file_name)
            except Exception as e:
                errors.append((file_name, str(e)))

    logging.info(f"Copied files: {copied_files}")
    if errors:
        logging.error(f"Errors during copy: {errors}")

    return copied_files

In [8]:
import shutil

In [9]:
# Example of usage
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    root_path = "/content/drive/MyDrive/ai_projects/image_classification"
    source_directory = "/content/drive/MyDrive/portugal"

   # Create folders
    folders = create_folders(root_path)

    # Copy files to 'original' folder
    copied_files = copy_to_original(source_directory, folders['original'])

    print(f"Copied files: {copied_files}")

2025-01-19 16:26:05,171 - INFO - Project initialized with folders: ['/content/drive/MyDrive/ai_projects/image_classification/original', '/content/drive/MyDrive/ai_projects/image_classification/resized', '/content/drive/MyDrive/ai_projects/image_classification/converted', '/content/drive/MyDrive/ai_projects/image_classification/cleaned', '/content/drive/MyDrive/ai_projects/image_classification/results']
2025-01-19 16:28:19,199 - INFO - Copied files: ['IMG_7961.JPEG', 'IMG_7962.JPEG', 'IMG_7954.JPEG', 'IMG_7963.JPEG', 'IMG_7964.JPEG', 'IMG_7952.JPEG', 'IMG_7927.HEIC', 'IMG_7926.HEIC', 'IMG_7925.HEIC', 'IMG_7923.HEIC', 'IMG_7920.JPEG', 'IMG_7914.JPEG', 'IMG_7912.HEIC', 'IMG_7909.JPEG', 'IMG_7897.JPEG', 'IMG_7900.JPEG', 'IMG_7886.JPEG', 'IMG_7894.JPEG', 'IMG_7888.JPEG', 'IMG_7883.JPEG', 'IMG_7877.JPEG', 'IMG_7884.JPEG', 'IMG_7874.JPEG', 'IMG_7872.JPEG', 'IMG_7869.JPEG', 'IMG_7876.HEIC', 'IMG_7866.JPEG', 'IMG_7859.JPEG', 'IMG_7862.JPEG', 'IMG_7854.JPEG', 'IMG_7855.JPEG', 'IMG_7844.JPEG', 'I

Copied files: ['IMG_7961.JPEG', 'IMG_7962.JPEG', 'IMG_7954.JPEG', 'IMG_7963.JPEG', 'IMG_7964.JPEG', 'IMG_7952.JPEG', 'IMG_7927.HEIC', 'IMG_7926.HEIC', 'IMG_7925.HEIC', 'IMG_7923.HEIC', 'IMG_7920.JPEG', 'IMG_7914.JPEG', 'IMG_7912.HEIC', 'IMG_7909.JPEG', 'IMG_7897.JPEG', 'IMG_7900.JPEG', 'IMG_7886.JPEG', 'IMG_7894.JPEG', 'IMG_7888.JPEG', 'IMG_7883.JPEG', 'IMG_7877.JPEG', 'IMG_7884.JPEG', 'IMG_7874.JPEG', 'IMG_7872.JPEG', 'IMG_7869.JPEG', 'IMG_7876.HEIC', 'IMG_7866.JPEG', 'IMG_7859.JPEG', 'IMG_7862.JPEG', 'IMG_7854.JPEG', 'IMG_7855.JPEG', 'IMG_7844.JPEG', 'IMG_7838.JPEG', 'IMG_7836.JPEG', 'IMG_7840.JPEG', 'IMG_7835.JPEG', 'IMG_7831.JPEG', 'IMG_7824.JPEG', 'IMG_7825.HEIC', 'IMG_7827.HEIC', 'IMG_7822.JPEG', 'IMG_7820.PNG', 'IMG_7810.JPEG', 'IMG_7809.HEIC', 'IMG_7774.JPEG', 'IMG_7799.JPEG', 'IMG_7788.JPEG', 'IMG_7786.JPEG', 'IMG_7769.JPEG', 'IMG_7763.JPEG', 'IMG_7760.HEIC', 'IMG_7755.HEIC', 'IMG_7758.JPEG', 'IMG_7753.HEIC', 'IMG_7751.PNG', 'IMG_7750.JPEG', 'IMG_7754.HEIC', 'IMG_7749.JPEG', '

In [10]:
pip install pillow pyheif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 15.9 MB/s eta 0:00:00


In [11]:
from PIL import Image
import pyheif

import os
import logging
import pyheif
from PIL import Image

def heic_to_rgb(image_path, output_path, target_format='jpeg'):
    """
    Convert a HEIC image to the specified format.

    Parameters:
        image_path (str): Path to the HEIC image.
        output_path (str): Path to save the converted image.
        target_format (str): Format to convert to (default: 'jpeg').
    """
    heif_file = pyheif.read(image_path)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )
    image.convert("RGB").save(output_path, format=target_format.upper())

def process_images(source_folder, destination_folder, target_format='jpeg'):
    """
    Process images from a source folder and copy them to a destination folder.

    The function handles HEIC conversion to a specified target format and ensures
    that all valid image files are processed. Non-image files are skipped.
    Converted HEIC files are moved to a 'converted' folder.

    Parameters:
        source_folder (str): Folder containing the source images.
        destination_folder (str): Folder where processed images will be saved.
        target_format (str): Target format for image conversion (default: 'jpeg').

    Returns:
        None
    """
    try:
        # Ensure the destination folder is clean
        if os.path.exists(destination_folder):
            for file_name in os.listdir(destination_folder):
                file_path = os.path.join(destination_folder, file_name)
                try:
                    os.remove(file_path)
                except Exception as e:
                    logging.warning(f"Failed to delete file {file_name}: {e}")
        else:
            os.makedirs(destination_folder, exist_ok=True)

        # Ensure the converted folder exists
        converted_folder = os.path.join(destination_folder, "converted")
        os.makedirs(converted_folder, exist_ok=True)

        # Initialize counters
        copied_count = 0
        converted_count = 0
        skipped_count = 0

        # Process files in the source folder
        for file_name in os.listdir(source_folder):
            source_path = os.path.join(source_folder, file_name)
            destination_path = os.path.join(destination_folder, f"{os.path.splitext(file_name)[0]}.{target_format}")

            try:
                # Handle HEIC conversion
                if file_name.lower().endswith('.heic'):
                    heic_to_rgb(source_path, destination_path, target_format)
                    shutil.move(destination_path, os.path.join(converted_folder, os.path.basename(destination_path)))
                    converted_count += 1
                else:
                    # Copy other image formats directly
                    with Image.open(source_path) as img:
                        img.convert("RGB").save(destination_path, format=target_format.upper())
                    copied_count += 1
            except Exception as e:
                # Count files that could not be processed
                logging.warning(f"Skipped file {file_name}: {e}")
                skipped_count += 1

        # Log summary
        logging.info(f"Summary of image processing:")
        logging.info(f" - Copied without conversion: {copied_count}")
        logging.info(f" - Converted from HEIC to {target_format.upper()}: {converted_count}")
        logging.info(f" - Skipped files (not processed): {skipped_count}")

    except Exception as e:
        logging.error(f"An error occurred during image processing: {e}")

In [12]:
import logging

if __name__ == "__main__":
    # Configuração do logging
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

    # Definição de diretórios
    source_folder = "/content/drive/MyDrive/ai_projects/image_classification/original"  # Substitua pelo caminho da pasta de origem
    destination_folder = "/content/drive/MyDrive/ai_projects/image_classification/converted"  # Substitua pelo caminho da pasta de destino

    # Definição do formato alvo
    target_format = "jpeg"  # Pode ser alterado para outros formatos suportados como 'png', 'bmp', etc.

    # Execução do processamento de imagens
    logging.info("Iniciando o processamento de imagens...")
    process_images(source_folder, destination_folder, target_format)
    logging.info("Processamento de imagens concluído!")

2025-01-19 16:28:27,012 - INFO - Iniciando o processamento de imagens...
2025-01-19 16:32:29,515 - WARNING - Skipped file 10329FC9-C889-4E24-A687-3EE379475126.mp4: cannot identify image file '/content/drive/MyDrive/ai_projects/image_classification/original/10329FC9-C889-4E24-A687-3EE379475126.mp4'
2025-01-19 16:35:41,828 - INFO - Summary of image processing:
2025-01-19 16:35:41,832 - INFO -  - Copied without conversion: 431
2025-01-19 16:35:41,833 - INFO -  - Converted from HEIC to JPEG: 132
2025-01-19 16:35:41,836 - INFO -  - Skipped files (not processed): 1
2025-01-19 16:35:41,876 - INFO - Processamento de imagens concluído!


In [13]:
def rename_images_in_folder(folder_path, mapping_path):
    """
    Rename images in the specified folder sequentially and save the mapping to a CSV file.

    Parameters:
        folder_path (str): Path to the folder containing the images.
        mapping_path (str): Path to save the name mapping CSV.

    Returns:
        pd.DataFrame: DataFrame containing the mapping of original to renamed files.
    """
    if not os.path.exists(folder_path):
        logging.error(f"Folder not found: {folder_path}")
        raise FileNotFoundError(f"Folder not found: {folder_path}")

    files = [f for f in os.listdir(folder_path) if f.lower().endswith(('jpg', 'jpeg', 'png', 'bmp', 'tiff'))]
    if not files:
        logging.warning("No image files found in the folder.")
        return pd.DataFrame(columns=["original_name", "file_name"])

    files.sort()
    mapping = []

    for idx, file_name in enumerate(files):
        old_path = os.path.join(folder_path, file_name)
        new_name = f"{idx + 1}{os.path.splitext(file_name)[1]}"
        new_path = os.path.join(folder_path, new_name)
        try:
            os.rename(old_path, new_path)
            mapping.append({"original_name": file_name, "file_name": new_name})
        except Exception as e:
            logging.error(f"Error renaming {file_name}: {e}")

    mapping_df = pd.DataFrame(mapping)
    try:
        mapping_df.to_csv(mapping_path, index=False)
        logging.info(f"Name mapping saved to {mapping_path}")
    except Exception as e:
        logging.error(f"Error saving name mapping CSV: {e}")
        raise e

    return mapping_df

def resize_images(folder_path, output_path, size=(256, 256)):
    """
    Resize images in a folder to the specified size and save them to the output folder.

    Parameters:
        folder_path (str): Path to the folder containing images to resize.
        output_path (str): Path to the folder to save resized images.
        size (tuple): Desired size for the resized images (width, height).

    Returns:
        None
    """
    logging.info("Starting resizing process...")

    # Clear the output folder before resizing
    if os.path.exists(output_path):
        for file_name in os.listdir(output_path):
            file_path = os.path.join(output_path, file_name)
            try:
                os.remove(file_path)
            except Exception as e:
                logging.error(f"Failed to delete {file_name}: {e}")
    else:
        os.makedirs(output_path, exist_ok=True)

    files = os.listdir(folder_path)
    if not files:
        logging.warning("No files found to resize. Please check the path.")
        return

    resized_count = 0
    failed_count = 0

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        try:
            img = Image.open(file_path)
            img_resized = img.resize(size)
            img_resized.save(os.path.join(output_path, file_name))

            logging.info(f"Resized: {file_name}")
            resized_count += 1
        except Exception as e:
            logging.error(f"Error resizing image {file_name}: {e}")
            failed_count += 1

    logging.info(f"Resizing process completed: {resized_count} files resized, {failed_count} files failed.")

In [14]:
import logging

if __name__ == "__main__":
    # Configurar logging para exibir informações no console
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")



    source_folder = "/content/drive/MyDrive/ai_projects/image_classification/original"  # Substitua pelo caminho da pasta de origem
    destination_folder = "/content/drive/MyDrive/ai_projects/image_classification/converted"  # Substitua pelo caminho da pasta de destino



    # Caminhos para as pastas e arquivos
    source_folder = "/content/drive/MyDrive/ai_projects/image_classification/converted"  # Substitua pelo caminho da pasta de origem
    renamed_mapping_path = "/content/drive/MyDrive/ai_projects/image_classification/renamed_mapping.csv"  # Caminho para salvar o mapeamento de nomes
    resized_folder = "/content/drive/MyDrive/ai_projects/image_classification/resized"  # Pasta para salvar as imagens redimensionadas
    resize_size = (256, 256)  # Tamanho desejado para as imagens redimensionadas (largura, altura)

    try:
        # Etapa 1: Renomear imagens sequencialmente
        logging.info("Renomeando imagens na pasta de origem...")
        rename_mapping_df = rename_images_in_folder(source_folder, renamed_mapping_path)
        logging.info(f"Renomeação concluída. Mapeamento salvo em: {renamed_mapping_path}")

        # Etapa 2: Redimensionar as imagens renomeadas
        logging.info("Redimensionando imagens para o tamanho especificado...")
        resize_images(source_folder, resized_folder, size=resize_size)
        logging.info("Redimensionamento concluído. Imagens salvas na pasta de destino.")
    except Exception as e:
        logging.error(f"Erro durante o processamento: {e}")

2025-01-19 16:35:41,912 - INFO - Renomeando imagens na pasta de origem...
2025-01-19 16:35:43,179 - INFO - Name mapping saved to /content/drive/MyDrive/ai_projects/image_classification/renamed_mapping.csv
2025-01-19 16:35:43,185 - INFO - Renomeação concluída. Mapeamento salvo em: /content/drive/MyDrive/ai_projects/image_classification/renamed_mapping.csv
2025-01-19 16:35:43,187 - INFO - Redimensionando imagens para o tamanho especificado...
2025-01-19 16:35:43,191 - INFO - Starting resizing process...
2025-01-19 16:35:43,218 - ERROR - Error resizing image converted: [Errno 21] Is a directory: '/content/drive/MyDrive/ai_projects/image_classification/converted/converted'
2025-01-19 16:35:43,576 - INFO - Resized: 405.jpeg
2025-01-19 16:35:43,939 - INFO - Resized: 406.jpeg
2025-01-19 16:35:45,256 - INFO - Resized: 403.jpeg
2025-01-19 16:35:45,655 - INFO - Resized: 407.jpeg
2025-01-19 16:35:46,077 - INFO - Resized: 408.jpeg
2025-01-19 16:35:46,416 - INFO - Resized: 401.jpeg
2025-01-19 16:35

In [15]:
pip install imagehash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.6 MB/s eta 0:00:00


In [30]:
import shutil
import pandas as pd
import os
from itertools import combinations
from skimage.metrics import structural_similarity as ssim
import cv2


def calculate_similarity(image1_path, image2_path):
    """
    Calculate the Structural Similarity Index (SSIM) between two images.

    Parameters:
        image1_path (str): Path to the first image.
        image2_path (str): Path to the second image.

    Returns:
        float: SSIM similarity score (0 to 1).
    """
    try:
        image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
        image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)
        if image1 is None or image2 is None:
            return 0.0
        image1 = cv2.resize(image1, (256, 256))
        image2 = cv2.resize(image2, (256, 256))
        score, _ = ssim(image1, image2, full=True)
        return score
    except Exception as e:
        logging.error(f"Error calculating similarity: {e}")
        return 0.0


def calculate_sharpness(image_path):
    """
    Calculate the sharpness of an image using the Laplacian variance.

    Parameters:
        image_path (str): Path to the image.

    Returns:
        float: Sharpness score (higher is sharper).
    """
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            return 0.0
        return cv2.Laplacian(image, cv2.CV_64F).var()
    except Exception as e:
        logging.error(f"Error calculating sharpness: {e}")
        return 0.0


def calculate_exposure(image_path):
    """
    Calculate the average brightness of an image.

    Parameters:
        image_path (str): Path to the image.

    Returns:
        float: Exposure score (0 to 1, where 0.5 is ideal exposure).
    """
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            return 0.0
        return image.mean() / 255.0
    except Exception as e:
        logging.error(f"Error calculating exposure: {e}")
        return 0.0


import cv2
import numpy as np

def calculate_hash(image_path):
    """
    Calculate a perceptual hash (dHash) for an image.

    Parameters:
        image_path (str): Path to the image file.

    Returns:
        str: A hash value representing the image.
    """
    try:
        # Load the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError("Image not found or unable to read.")

        # Resize to 9x8 to calculate differences
        resized = cv2.resize(image, (9, 8), interpolation=cv2.INTER_AREA)

        # Compute the differences between adjacent pixels
        diff = resized[:, 1:] > resized[:, :-1]

        # Convert differences to a hash value
        hash_val = ''.join(str(int(x)) for x in diff.flatten())
        return hash_val
    except Exception as e:
        logging.error(f"Error calculating hash for {image_path}: {e}")
        return None


def evaluate_images_from_folders(resized_folder, cleaned_folder, output_csv, criteria):
    """
    Evaluate images based on similarity, sharpness, and exposure, and move rejected images.
    """
    logging.info("Starting evaluation of images from folders...")
    deletion_log = []

    # List images in folder
    image_files = [
        f for f in os.listdir(resized_folder)
        if f.lower().endswith(('jpg', 'jpeg', 'png', 'bmp', 'tiff'))
    ]
    image_paths = {img: os.path.join(resized_folder, img) for img in image_files}

    # Step 1: Detect duplicates using SSIM instead of hash
    logging.info("Step 1: Detecting duplicates with SSIM...")
    checked_pairs = set()
    duplicates_groups = {}

    for img1, img2 in combinations(image_files, 2):
        pair = tuple(sorted([img1, img2]))
        if pair in checked_pairs:
            continue

        sim_score = calculate_similarity(image_paths[img1], image_paths[img2])
        if sim_score > criteria["similarity_threshold"]:  # Adjusted threshold
            if pair[0] not in duplicates_groups:
                duplicates_groups[pair[0]] = [pair[1]]
            else:
                duplicates_groups[pair[0]].append(pair[1])

        checked_pairs.add(pair)

    # Evaluate duplicates and move lower-quality images
    for main_image, duplicates in duplicates_groups.items():
        best_image = main_image
        best_score = calculate_sharpness(image_paths[main_image]) - abs(calculate_exposure(image_paths[main_image]) - 0.5)

        for image_name in duplicates:
            sharpness_score = calculate_sharpness(image_paths[image_name])
            exposure_score = calculate_exposure(image_paths[image_name])
            total_score = sharpness_score - abs(exposure_score - 0.5)

            if total_score > best_score:
                best_score = total_score
                best_image = image_name

        # Move all duplicates except the best image
        for image_name in duplicates:
            if image_name != best_image:
                shutil.move(image_paths[image_name], os.path.join(cleaned_folder, image_name))
                logging.info(f"Moved duplicate: {image_name}")
                deletion_log.append({"Image Name": image_name, "Reason": "Duplicate SSIM"})

    # Step 2: Evaluate sharpness and exposure for non-duplicates
    logging.info("Step 2: Evaluating sharpness and exposure...")
    remaining_files = [f for f in image_files if f not in duplicates_groups]

    for image_name in remaining_files:
        sharpness_score = calculate_sharpness(image_paths[image_name])
        if sharpness_score < criteria["sharpness_threshold"]:
            shutil.move(image_paths[image_name], os.path.join(cleaned_folder, image_name))
            logging.info(f"Moved: {image_name} due to low sharpness")
            deletion_log.append({"Image Name": image_name, "Reason": "Low sharpness"})

    # Exposure evaluation
    for image_name in remaining_files:
        exposure_score = calculate_exposure(image_paths[image_name])
        if abs(exposure_score - 0.5) > criteria["exposure_tolerance"]:
            shutil.move(image_paths[image_name], os.path.join(cleaned_folder, image_name))
            logging.info(f"Moved: {image_name} due to poor exposure")
            deletion_log.append({"Image Name": image_name, "Reason": "Poor exposure"})

    # Save deletion log
    if deletion_log:
        pd.DataFrame(deletion_log).to_csv(output_csv, index=False)
    logging.info(f"Deletion log saved to {output_csv}")
    return pd.DataFrame(deletion_log)

In [31]:
# Example usage
resized_folder = "/content/drive/MyDrive/ai_projects/image_classification/resized"
cleaned_folder = "/content/drive/MyDrive/ai_projects/image_classification/cleaned"
output_csv = "/content/drive/MyDrive/ai_projects/image_classification/deletion_log.csv"

# Configurable criteria
CRITERIA = {
    "similarity_threshold": 0.7,  # Threshold for similarity
    "sharpness_threshold": 100,   # Minimum sharpness score
    "exposure_tolerance": 0.2    # Exposure deviation from ideal (0.5)
}

# Evaluate images
deletion_log_df = evaluate_images_from_folders(resized_folder, cleaned_folder, output_csv, CRITERIA)

# Print deletion log
print(deletion_log_df)

2025-01-19 17:50:59,847 - INFO - Starting evaluation of images from folders...
2025-01-19 17:50:59,861 - INFO - Step 1: Detecting duplicates with SSIM...
2025-01-19 18:15:16,561 - INFO - Moved duplicate: 240.jpeg
2025-01-19 18:15:16,616 - INFO - Step 2: Evaluating sharpness and exposure...
2025-01-19 18:15:21,841 - INFO - Deletion log saved to /content/drive/MyDrive/ai_projects/image_classification/deletion_log.csv


  Image Name          Reason
0   240.jpeg  Duplicate SSIM


In [25]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-tgvhgvgp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-tgvhgvgp
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=4277e877f3cb4b09053072bc029435ce47ab86bd05a99ab4a00739de7271867f
  Stored in directory: /tmp/pip-ephem-wheel-cache-rzh5nuy_/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [28]:
import os
import logging
import numpy as np
import pandas as pd
import torch
from torchvision import models, transforms
from tensorflow.keras.applications import (
    MobileNetV3Large, InceptionResNetV2, ResNet101, EfficientNetB7, DenseNet201
)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mobilenetv3_preprocess
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as inceptionresnet_preprocess
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

import clip
from PIL import Image
import torch

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def get_model_map(framework="tensorflow"):
    if framework == "tensorflow":
        model_map = {
            "MobileNetV3": (MobileNetV3Large(weights="imagenet", include_top=False, pooling="avg"), mobilenetv3_preprocess),
            "InceptionResNetV2": (InceptionResNetV2(weights="imagenet", include_top=False, pooling="avg"), inceptionresnet_preprocess),
            "ResNet101": (ResNet101(weights="imagenet", include_top=False, pooling="avg"), resnet_preprocess),
            "EfficientNetB7": (EfficientNetB7(weights="imagenet", include_top=False, pooling="avg"), efficientnet_preprocess),
            "DenseNet201": (DenseNet201(weights="imagenet", include_top=False, pooling="avg"), densenet_preprocess),
        }
    elif framework == "pytorch":
        preprocess = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        model_map = {
            "resnet18": (models.resnet18(pretrained=True), preprocess),
            "resnet34": (models.resnet34(pretrained=True), preprocess),
            "efficientnet_b0": (models.efficientnet_b0(pretrained=True), preprocess),
            "vision_transformer": (models.vit_b_16(pretrained=True), preprocess),
            "clip": (clip.load("ViT-B/32")[0], clip.load("ViT-B/32")[1]),  # CLIP Model
        }
    else:
        raise ValueError("Unsupported framework. Choose 'tensorflow' or 'pytorch'.")
    return model_map

def process_image(image_path, model_map, framework="tensorflow"):
    try:
        scores = {"file_name": os.path.basename(image_path)}

        if framework == "tensorflow":
            img = image.load_img(image_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)

            for model_name, (model, preprocess) in model_map.items():
                img_preprocessed = preprocess(img_array)
                features = model.predict(img_preprocessed)
                scores[model_name] = np.linalg.norm(features)

        elif framework == "pytorch":
            img = Image.open(image_path).convert("RGB")
            for model_name, (model, preprocess) in model_map.items():
                if model_name == "clip":
                    img_preprocessed = preprocess(img).unsqueeze(0)
                    model.eval()
                    with torch.no_grad():
                        text = clip.tokenize(["aesthetic photo", "high-quality image"])
                        image_features = model.encode_image(img_preprocessed)
                        text_features = model.encode_text(text)
                        similarity = torch.cosine_similarity(image_features, text_features)
                        scores[model_name] = similarity[0].item()
                else:
                    img_preprocessed = preprocess(img).unsqueeze(0)
                    model.eval()
                    with torch.no_grad():
                        features = model(img_preprocessed).flatten()
                    scores[model_name] = features.norm().item()

        return scores
    except Exception as e:
        logging.error(f"Error processing image {image_path}: {e}")
        return None

def predict_and_generate_log(root_folder, framework="tensorflow", model_name=None):
    """
    Predict image scores using selected models, save results to log CSV,
    and add rankings for each model.

    Parameters:
        root_folder (str): Root directory of the project.
        framework (str): Framework used for models ('tensorflow' or 'pytorch').
        model_name (str): Specific model to run (e.g., 'clip', 'ResNet101'). If None, runs all models.
    """
    resized_folder = os.path.join(root_folder, "resized")

    # Load image files
    image_files = [
        os.path.join(resized_folder, f) for f in os.listdir(resized_folder)
        if f.lower().endswith(("jpg", "jpeg", "png", "bmp"))
    ]
    if not image_files:
        logging.warning(f"No images found in the resized folder for {framework}.")
        return

    # Load all models and preprocessors
    model_map = get_model_map(framework)

    # If a specific model is requested, filter the model map
    if model_name and model_name in model_map:
        model_map = {model_name: model_map[model_name]}
        csv_file_suffix = f"_{model_name}"
    else:
        csv_file_suffix = f"_{framework}"

    logging.info(f"Processing with models: {list(model_map.keys())} using {framework}")

    results = []

    # Process images
    for idx, image_path in enumerate(image_files):
        scores = process_image(image_path, model_map, framework)
        if scores:
            results.append(scores)
        if (idx + 1) % 10 == 0 or (idx + 1) == len(image_files):
            logging.info(f"Processed {idx + 1}/{len(image_files)} images using {framework}.")

    # Convert to DataFrame and save
    df = pd.DataFrame(results)

    # Ensure all model columns are present
    for model in model_map.keys():
        if model not in df.columns:
            df[model] = None

    # Define the output CSV filename based on framework or model name
    scores_csv_path = os.path.join(root_folder, f"model_scores{csv_file_suffix}.csv")

    # Add rankings for each model
    for model in model_map.keys():
        if df[model].notnull().any():
            df[f"{model}_rank"] = df[model].rank(ascending=False, method="min")

    df.to_csv(scores_csv_path, index=False)
    logging.info(f"Model scores with rankings saved to {scores_csv_path}")


In [21]:
import os
import pandas as pd

# Define the root folder containing the "resized" folder with images
root_folder = "/content/drive/MyDrive/ai_projects/image_classification"

# Step 1: Run the prediction and ranking process for TensorFlow and PyTorch models separately
print("Running TensorFlow-based predictions...")
predict_and_generate_log(root_folder, framework="tensorflow")

print("Running PyTorch-based predictions...")
predict_and_generate_log(root_folder, framework="pytorch")

# Step 2: Load and display results from both CSVs
tf_csv_path = os.path.join(root_folder, "model_scores_tensorflow.csv")
pt_csv_path = os.path.join(root_folder, "model_scores_pytorch.csv")

# Function to display top ranked images for each model
def display_top_images(csv_path, framework_name):
    if os.path.exists(csv_path):
        print(f"\n{framework_name} Results:")
        scores_df = pd.read_csv(csv_path)

        # Identify model ranking columns (ending in '_rank')
        ranking_columns = [col for col in scores_df.columns if col.endswith('_rank')]

        if ranking_columns:
            for rank_col in ranking_columns:
                model_name = rank_col.replace("_rank", "")
                print(f"\nTop 20 Images Ranked by {model_name} ({framework_name}):")
                top_images = scores_df.sort_values(by=rank_col).head(20)
                print(top_images[["file_name", rank_col]])
        else:
            print(f"No ranking columns found in {framework_name} results.")
    else:
        print(f"Scores file not found at {csv_path}.")

# Display results for TensorFlow models
display_top_images(tf_csv_path, "TensorFlow")

# Display results for PyTorch models
display_top_images(pt_csv_path, "PyTorch")

Running TensorFlow-based predictions...


/usr/local/lib/python3.11/dist-packages/keras/src/applications/mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


2025-01-19 16:44:02,431 - INFO - Processing with models: ['MobileNetV3', 'InceptionResNetV2', 'ResNet101', 'EfficientNetB7', 'DenseNet201'] using tensorflow


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step


2025-01-19 16:56:56,755 - INFO - Processed 381/381 images using tensorflow.
2025-01-19 16:56:56,795 - INFO - Model scores with rankings saved to /content/drive/MyDrive/ai_projects/image_classification/model_scores_tensorflow.csv


Running PyTorch-based predictions...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 112MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i


TensorFlow Results:

Top 20 Images Ranked by MobileNetV3 (TensorFlow):
    file_name  MobileNetV3_rank
68   195.jpeg               1.0
70   194.jpeg               2.0
67   196.jpeg               3.0
114  387.jpeg               4.0
280  197.jpeg               5.0
55     1.jpeg               6.0
279  198.jpeg               7.0
30   310.jpeg               8.0
110    4.jpeg               9.0
59     6.jpeg              10.0
277  203.jpeg              11.0
66   200.jpeg              12.0
217  282.jpeg              13.0
69   193.jpeg              14.0
205  105.jpeg              15.0
180  171.jpeg              16.0
26   318.jpeg              17.0
284  162.jpeg              18.0
278  199.jpeg              19.0
80   157.jpeg              20.0

Top 20 Images Ranked by InceptionResNetV2 (TensorFlow):
    file_name  InceptionResNetV2_rank
241  308.jpeg                     1.0
279  198.jpeg                     2.0
69   193.jpeg                     3.0
280  197.jpeg                     4.0
140  307.

In [22]:
import os
import shutil
import logging
import pandas as pd

def transfer_top_images_by_framework(tf_csv_path, pt_csv_path, name_mapping_path, original_folder, results_folder, num_images=20):
    """
    Transfer top-ranked original images to subfolders in the results folder for each framework (TensorFlow/PyTorch).

    Parameters:
        tf_csv_path (str): Path to the TensorFlow scores CSV.
        pt_csv_path (str): Path to the PyTorch scores CSV.
        name_mapping_path (str): Path to the name mapping CSV (resized-to-original mapping).
        original_folder (str): Path to the folder containing original images.
        results_folder (str): Path to save the selected images.
        num_images (int): Number of top-ranked images to transfer for each model.
    """
    # Create framework-specific folders
    tensorflow_results = os.path.join(results_folder, "tensorflow_models")
    pytorch_results = os.path.join(results_folder, "pytorch_models")
    os.makedirs(tensorflow_results, exist_ok=True)
    os.makedirs(pytorch_results, exist_ok=True)

    # Load the name mapping CSV and normalize names
    mapping_df = pd.read_csv(name_mapping_path)
    mapping_df["original_name_normalized"] = mapping_df["original_name"].str.strip().str.lower()
    mapping_df["file_name"] = mapping_df["file_name"].str.strip().str.lower()

    # Index available files in the original folder
    available_files = {
        f.lower(): os.path.abspath(os.path.join(original_folder, f))
        for f in os.listdir(original_folder)
    }
    logging.info(f"Available files in original folder: {list(available_files.keys())}")

    # Process both TensorFlow and PyTorch models
    for framework, csv_path, target_folder in [
        ("TensorFlow", tf_csv_path, tensorflow_results),
        ("PyTorch", pt_csv_path, pytorch_results)
    ]:
        if not os.path.exists(csv_path):
            logging.warning(f"CSV file for {framework} not found: {csv_path}")
            continue

        # Load scores and normalize file names
        scores_df = pd.read_csv(csv_path)
        scores_df["file_name"] = scores_df["file_name"].str.strip().str.lower()

        # Merge scores and mappings
        merged_df = pd.merge(scores_df, mapping_df, how="left", on="file_name")
        if "original_name_normalized" not in merged_df.columns:
            raise ValueError(f"Merged DataFrame for {framework} is missing 'original_name_normalized' column.")

        # Identify ranking columns (assume columns ending with '_rank')
        ranking_columns = [col for col in scores_df.columns if col.endswith("_rank")]
        logging.info(f"Ranking columns identified for {framework}: {ranking_columns}")

        # Process each ranking column (model)
        for ranking_column in ranking_columns:
            model_name = ranking_column.replace("_rank", "")
            model_folder = os.path.join(target_folder, model_name)
            os.makedirs(model_folder, exist_ok=True)
            logging.info(f"Processing model {model_name} under {framework}")

            # Select top-ranked images for the model
            top_images = merged_df.sort_values(by=ranking_column, ascending=True).head(num_images)
            transferred_files = []

            # Transfer images
            for _, row in top_images.iterrows():
                original_name_normalized = row["original_name_normalized"]
                original_path = available_files.get(original_name_normalized)

                if original_path:
                    try:
                        destination_path = os.path.join(model_folder, os.path.basename(original_path))
                        shutil.copy(original_path, destination_path)
                        transferred_files.append(original_name_normalized)
                        logging.info(f"Transferred: {os.path.basename(original_path)} to {model_folder}")
                    except Exception as e:
                        logging.error(f"Error transferring {os.path.basename(original_path)}: {e}")
                else:
                    logging.warning(f"File not found: {original_name_normalized}")

            # Save transfer log for the model
            transfer_log_path = os.path.join(model_folder, "transfer_log.csv")
            top_images["transferred"] = top_images["original_name_normalized"].isin(transferred_files)
            top_images.to_csv(transfer_log_path, index=False)
            logging.info(f"Transfer log saved for model {model_name} at {transfer_log_path}")

In [23]:
# Define paths
root_folder = "/content/drive/MyDrive/ai_projects/image_classification"
tf_csv_path = os.path.join(root_folder, "model_scores_tensorflow.csv")
pt_csv_path = os.path.join(root_folder, "model_scores_pytorch.csv")
name_mapping_path = os.path.join(root_folder, "renamed_mapping.csv")
original_folder = os.path.join(root_folder, "original")
results_folder = os.path.join(root_folder, "results")

# Run transfer function for top 20 images per model
transfer_top_images_by_framework(
    tf_csv_path=tf_csv_path,
    pt_csv_path=pt_csv_path,
    name_mapping_path=name_mapping_path,
    original_folder=original_folder,
    results_folder=results_folder,
    num_images=20
)

2025-01-19 17:21:57,720 - INFO - Available files in original folder: ['img_7961.jpeg', 'img_7962.jpeg', 'img_7954.jpeg', 'img_7963.jpeg', 'img_7964.jpeg', 'img_7952.jpeg', 'img_7927.heic', 'img_7926.heic', 'img_7925.heic', 'img_7923.heic', 'img_7920.jpeg', 'img_7914.jpeg', 'img_7912.heic', 'img_7909.jpeg', 'img_7897.jpeg', 'img_7900.jpeg', 'img_7886.jpeg', 'img_7894.jpeg', 'img_7888.jpeg', 'img_7883.jpeg', 'img_7877.jpeg', 'img_7884.jpeg', 'img_7874.jpeg', 'img_7872.jpeg', 'img_7869.jpeg', 'img_7876.heic', 'img_7866.jpeg', 'img_7859.jpeg', 'img_7862.jpeg', 'img_7854.jpeg', 'img_7855.jpeg', 'img_7844.jpeg', 'img_7838.jpeg', 'img_7836.jpeg', 'img_7840.jpeg', 'img_7835.jpeg', 'img_7831.jpeg', 'img_7824.jpeg', 'img_7825.heic', 'img_7827.heic', 'img_7822.jpeg', 'img_7820.png', 'img_7810.jpeg', 'img_7809.heic', 'img_7774.jpeg', 'img_7799.jpeg', 'img_7788.jpeg', 'img_7786.jpeg', 'img_7769.jpeg', 'img_7763.jpeg', 'img_7760.heic', 'img_7755.heic', 'img_7758.jpeg', 'img_7753.heic', 'img_7751.png

In [29]:
# Definição dos caminhos
root_folder = "/content/drive/MyDrive/ai_projects/image_classification"
image_folder = os.path.join(root_folder, "resized")
clip_csv_path = os.path.join(root_folder, "model_scores_clip.csv")
name_mapping_path = os.path.join(root_folder, "renamed_mapping.csv")
original_folder = os.path.join(root_folder, "original")
clip_results_folder = os.path.join(root_folder, "results/clip_model")

# Etapa 1: Executar a predição somente para o modelo CLIP
print("Running CLIP-based predictions...")
predict_and_generate_log(root_folder, framework="pytorch", model_name="clip")

# Etapa 2: Transferir as 20 melhores imagens ranqueadas pelo CLIP
print("Transferring top 20 CLIP images...")
transfer_top_images_by_model(
    csv_path=clip_csv_path,
    name_mapping_path=name_mapping_path,
    original_folder=original_folder,
    results_folder=clip_results_folder,
    num_images=20
)

# Exibir log das imagens transferidas
if os.path.exists(clip_csv_path):
    clip_scores_df = pd.read_csv(clip_csv_path)
    print("\nTop 20 CLIP Ranked Images:")
    print(clip_scores_df.sort_values(by="clip_rank").head(20)[["file_name", "clip_rank"]])

Running CLIP-based predictions...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weig

Transferring top 20 CLIP images...


2025-01-19 17:34:57,049 - INFO - Transferred: IMG_7874.JPEG to /content/drive/MyDrive/ai_projects/image_classification/results/clip_model/clip
2025-01-19 17:34:57,114 - INFO - Transferred: IMG_7951.JPEG to /content/drive/MyDrive/ai_projects/image_classification/results/clip_model/clip
2025-01-19 17:34:57,201 - INFO - Transferred: IMG_7509.JPEG to /content/drive/MyDrive/ai_projects/image_classification/results/clip_model/clip
2025-01-19 17:35:00,273 - INFO - Transferred: IMG_7893.JPEG to /content/drive/MyDrive/ai_projects/image_classification/results/clip_model/clip
2025-01-19 17:35:00,348 - INFO - Transferred: IMG_7756.JPEG to /content/drive/MyDrive/ai_projects/image_classification/results/clip_model/clip
2025-01-19 17:35:00,430 - INFO - Transferred: IMG_7863.JPEG to /content/drive/MyDrive/ai_projects/image_classification/results/clip_model/clip
2025-01-19 17:35:00,531 - INFO - Transferred: IMG_7508.JPEG to /content/drive/MyDrive/ai_projects/image_classification/results/clip_model/clip


Top 20 CLIP Ranked Images:
    file_name  clip_rank
226  372.jpeg        1.0
218  399.jpeg        2.0
13   357.jpeg        3.0
108  400.jpeg        4.0
295  119.jpeg        5.0
105  373.jpeg        6.0
250  278.jpeg        7.0
230  347.jpeg        8.0
297  118.jpeg        9.0
234  341.jpeg       10.0
20   340.jpeg       11.0
126  351.jpeg       12.0
90   120.jpeg       13.0
8    374.jpeg       14.0
122  356.jpeg       15.0
203  121.jpeg       16.0
350   71.jpeg       17.0
123  349.jpeg       18.0
107  398.jpeg       19.0
277  203.jpeg       20.0
